<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
from io import BytesIO

import os
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import random_shift, img_to_array
import PIL.ImageOps

from skimage.io import imshow, imread
from skimage.color import rgb2grey
import skimage


Using TensorFlow backend.


In [2]:
TRAIN_DIRECTORY_PATH = '../data/drum_dataset/train_set/'
TEST_DIRECTORY_PATH = '../data/drum_dataset/test_set/'

NUM_CLASSES = 2
IMG_ROWS, IMG_COLS = 65, 65

In [3]:
x_train = []
x_test = []
y_train = []
y_test = []

for digit in range(NUM_CLASSES):
    directory = TRAIN_DIRECTORY_PATH + str(digit) + '/'
    files = next(os.walk(directory))[2]
    
    for filename in files:
        img = imread(directory + filename)
        img = skimage.img_as_float(img)
        imgs = [
            random_shift(img, wrg=0.1, hrg=0.3, row_axis=0,
                         col_axis=1, channel_axis=2, fill_mode='nearest') * 255
            for _ in range(3)]
        
        for img_ in imgs:
            shifted_img = img_.astype(np.uint8)
            shifted_img = rgb2grey(shifted_img)
            x_train.append(shifted_img)
            y_train.append(digit)

for digit in range(NUM_CLASSES):
    directory = TEST_DIRECTORY_PATH + str(digit) + '/'
    files = next(os.walk(directory))[2]
    
    for filename in files:
        img = imread(directory + filename)
        img = skimage.img_as_float(img)
        imgs = [
            random_shift(img, wrg=0.1, hrg=0.3, row_axis=0,
                         col_axis=1, channel_axis=2, fill_mode='nearest') * 255
            for _ in range(3)]
        
        for img_ in imgs:
            shifted_img = img_.astype(np.uint8)
            shifted_img = rgb2grey(shifted_img)
            x_test.append(shifted_img)
            y_test.append(digit)
        
x_train = np.asarray(x_train)
x_train = x_train.reshape(x_train.shape[0], IMG_ROWS, IMG_COLS, 1)

x_test = np.asarray(x_test)
x_test = x_test.reshape(x_test.shape[0], IMG_ROWS, IMG_COLS, 1)

input_shape = (IMG_ROWS, IMG_COLS, 1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

x_train shape: (14394, 65, 65, 1)
14394 train samples
1710 test samples


In [4]:
model = Sequential()
model.add(
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adadelta(),
    metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=100,
    verbose=1,
    validation_data=(x_test, y_test))

Train on 14394 samples, validate on 1710 samples
Epoch 1/100
14394/14394 [==============================] - 5s - loss: 0.1912 - acc: 0.9266 - val_loss: 0.0881 - val_acc: 0.9708
Epoch 2/100
14394/14394 [==============================] - 3s - loss: 0.0521 - acc: 0.9860 - val_loss: 0.1022 - val_acc: 0.9708
Epoch 3/100
14394/14394 [==============================] - 3s - loss: 0.0417 - acc: 0.9903 - val_loss: 0.0376 - val_acc: 0.9842
Epoch 4/100
14394/14394 [==============================] - 3s - loss: 0.0318 - acc: 0.9917 - val_loss: 0.0291 - val_acc: 0.9877
Epoch 5/100
14394/14394 [==============================] - 3s - loss: 0.0256 - acc: 0.9939 - val_loss: 0.0262 - val_acc: 0.9918
Epoch 6/100
14394/14394 [==============================] - 3s - loss: 0.0232 - acc: 0.9941 - val_loss: 0.0189 - val_acc: 0.9936
Epoch 7/100
14394/14394 [==============================] - 3s - loss: 0.0186 - acc: 0.9956 - val_loss: 0.0205 - val_acc: 0.9930
Epoch 8/100
14394/14394 [==============================

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.004444469933380452
Test accuracy: 0.9982456140350877


In [8]:
model.save('../taiko/image/drum_img_model.h5')